In [ ]:
# Evaluation on Test Set (Baseline or Best XGBoost)

In [ ]:
REGION = "<YOUR_AWS_REGION>"
BUCKET = "<YOUR_S3_BUCKET_NAME>"
PREFIX = "fraud"
MODEL_DATA_S3 = "<S3_PATH_TO_MODEL_ARTIFACT>.tar.gz"  # set after training


In [ ]:
import boto3, tarfile, os, io, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, roc_curve, confusion_matrix, classification_report
from joblib import load

s3 = boto3.client('s3', region_name=REGION)

# Download model artifact locally and extract
bucket_name = BUCKET
key = MODEL_DATA_S3.replace(f"s3://{bucket_name}/", "")
os.makedirs("model_artifact", exist_ok=True)
local_tar = "model.tar.gz"
s3.download_file(bucket_name, key, local_tar)
with tarfile.open(local_tar) as tar:
    tar.extractall("model_artifact")

# Load model.joblib
model = load("model_artifact/model.joblib")

# Load test set from S3
obj = s3.get_object(Bucket=BUCKET, Key=f"{PREFIX}/data/processed/test.csv")
test = pd.read_csv(io.BytesIO(obj["Body"].read()))
y = test["Class"].values
X = test.drop(columns=["Class"]).values

# Scores & metrics
proba = model.predict_proba(X)[:,1] if hasattr(model, "predict_proba") else model.predict(X)
pred = (proba >= 0.5).astype(int)

print("ROC-AUC:", roc_auc_score(y, proba))
print("PR-AUC:", average_precision_score(y, proba))
print(classification_report(y, pred, zero_division=0))
print("Confusion matrix:\n", confusion_matrix(y, pred))

# PR Curve
prec, rec, thr = precision_recall_curve(y, proba)
plt.figure()
plt.plot(rec, prec)
plt.title("Precision-Recall Curve")
plt.xlabel("Recall"); plt.ylabel("Precision")
plt.grid(True); plt.show()

# ROC Curve
fpr, tpr, _ = roc_curve(y, proba)
plt.figure()
plt.plot(fpr, tpr)
plt.title("ROC Curve")
plt.xlabel("FPR"); plt.ylabel("TPR")
plt.grid(True); plt.show()
